In [3]:
pip install torch torchvision transformers datasets peft


In [4]:
import torch
from torch.utils.data import DataLoader
from transformers import DebertaV2Tokenizer, DebertaV2ForSequenceClassification, AdamW, DataCollatorWithPadding
from datasets import load_dataset
from tqdm import tqdm
from peft import LoraConfig, get_peft_model

# Load the MRPC dataset
dataset = load_dataset('glue', 'mrpc')

# Load the tokenizer
tokenizer = DebertaV2Tokenizer.from_pretrained('microsoft/deberta-v3-large')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], padding='max_length', truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Ensure the labels are present and correctly formatted
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Data collator that will dynamically pad the inputs
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Create DataLoaders
train_dataloader = DataLoader(tokenized_datasets['train'], batch_size=8, shuffle=True, collate_fn=data_collator)
val_dataloader = DataLoader(tokenized_datasets['validation'], batch_size=8, collate_fn=data_collator)

# Device configuration
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Load a pretrained DeBERTa model for sequence classification
model = DebertaV2ForSequenceClassification.from_pretrained('microsoft/deberta-v3-large', num_labels=2)

# Define LoRA configuration
best_r = 4  # Example value, should be tuned
lora_config = LoraConfig(
    modules_to_save=['classifier', 'pooler'],
    task_type="SEQ_CLS",
    r=best_r,
    target_modules=["query_proj", "value_proj"],
    lora_alpha=64
)

# Integrate LoRA with the model
model = get_peft_model(model, lora_config)
model.to(device)

def train(model, train_dataloader, optimizer, device, criterion=torch.nn.CrossEntropyLoss()):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc="Training"):
        inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_dataloader)
    return average_loss

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

def evaluate(model, val_dataloader, device):
    model.eval()
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc="Evaluating"):
            inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
            labels = batch['labels'].to(device)

            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=-1)

            total_correct += (predictions == labels).sum().item()
            total_samples += labels.size(0)

    accuracy = total_correct / total_samples
    return accuracy

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    train_loss = train(model, train_dataloader, optimizer, device)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}")
    val_accuracy = evaluate(model, val_dataloader, device)
    print(f"Validation Accuracy: {val_accuracy:.4f}")




pytorch_model.bin:  41%|####      | 357M/874M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

Training: 100%|██████████| 459/459 [04:00<00:00,  1.91it/s]


Epoch 1/5, Train Loss: 0.6294



Evaluating: 100%|██████████| 51/51 [00:12<00:00,  3.95it/s]


Validation Accuracy: 0.6985


...

Validation Accuracy: 0.8824



Training: 100%|██████████| 459/459 [03:56<00:00,  1.94it/s]


Epoch 5/5, Train Loss: 0.2608



Evaluating: 100%|██████████| 51/51 [00:12<00:00,  3.96it/s]

Validation Accuracy: 0.8971
